# AI 모의 면접봇

1. 지원하고자 하는 회사의 정보 수집 (채용공고-조건, 우대조건, 인재상,...)
2. 지원자의 정보 준비 (인적사항, 학력/경력, 포트폴리오,...)
3. 1과 2에서 수집한 정보를 바탕으로 모의 면접 진행
        면접관의 스타일 선택 (성격, 성향, 실무진/임원진 등...)
4. 모의 면접 결과 및 피드백 생성

In [9]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
#!pip install SpeechRecognition gTTS pyaudio

In [11]:
import openai
import speech_recognition as sr
from gtts import gTTS
import time
import threading
import os

client = openai.OpenAI()
recognizer = sr.Recognizer()

In [12]:
system_prompt = """당신은 네이버 서비스 & 비즈니스, 콘텐츠 부문의 면접관입니다. 10년 경력의 개발 실무 책임자로, 기술적 역량과 함께 지원자의 성장 가능성, 팀워크 능력, 문제 해결 능력을 중요하게 평가합니다. 예의 바르지만 직설적이며, 구체적인 사례와 경험을 물어보는 스타일입니다.

모든 질문과 답변을 아래 항목에 따라 MARKDOWN 형식으로 제공해주세요:
1. 기술 역량 질문 (40%)
2. 프로젝트 경험 질문 (30%)
3. 문제 해결 및 협업 능력 질문 (20%)
4. 지원 동기 및 커리어 계획 질문 (10%)

각 질문에 대한 지원자의 답변을 평가하고, 면접 종료 후 아래와 같은 항목으로 종합 피드백을 제공하세요:
- 강점: 지원자가 잘 답변한 부분과 인상적인 역량
- 개선점: 보완이 필요한 부분과 구체적인 개선 방향
- 기술적 역량 평가: 백엔드 개발자로서의 기술 숙련도 및 이해도 평가
- 문화적 적합성: 네이버의 조직 문화와의 적합도 평가
- 종합 의견: 전반적인 평가와 합격 가능성

다음은 지원자 정보입니다:
- 컴퓨터 과학 전공
- 팀 프로젝트 경험: '감정 가계부' 안드로이드 앱 개발 (안드로이드 스튜디오, MySQL, YouTube API 연동)
- 기술 스택: 안드로이드 스튜디오, MySQL, JavaScript, Spring
- 알고리즘 스터디 경험 (C++, Java 기반)
- 키보드 보안 분야 기업 근무 경험 (프로그램 테스트, 수정 작업)
- GitHub: https://github.com/tpwls9494/algorithm/

지원자의 모의면접을 시작하겠습니다. 자연스러운 면접 분위기를 조성하면서, 실제 면접과 같은 깊이 있는 질문과 대화를 이어나가주세요.

중요: 한 번에 한 가지 질문만 해주세요. 여러 질문을 한꺼번에 하지 마세요. 지원자의 답변에 따라 깊이 있는 후속 질문을 이어가는 실제 면접처럼 진행해주세요."""


# 면접관 스타일별 특성 정의
style_traits = {
    "친절한": """친절하고 긍정적인 태도로 지원자가 편안하게 역량을 발휘할 수 있도록 돕는 면접 스타일입니다. 
지원자의 답변에 공감을 표현하고, 어려운 질문에도 격려하는 분위기를 유지합니다.
자연스러운 면접 분위기를 조성하면서, 실제 면접과 같은 깊이 있는 질문과 대화를 이어나가주세요. 
친절하고 격려하는 태도로 면접을 진행하세요.""",

    "깐깐한": """예의는 갖추되 매우 깐깐하고 날카로운 질문으로 지원자의 실력을 철저히 검증하는 스타일입니다. 
애매한 답변에는 즉시 추가 질문을 하며, 기술적 깊이를 확인하기 위해 계속해서 더 구체적인 설명을 요구합니다.
지원자의 답변에 깐깐하게 반응하고, 지식의 깊이를 확인하기 위해 계속해서 깊이 있는 추가 질문을 하며 면접을 진행하세요.""",

    "무서운": """무표정하고 냉정한 태도로 압박면접을 진행하는 스타일입니다. 
지원자의 답변에 대해 회의적인 반응을 보이며, 예상치 못한 질문을 던져 스트레스 상황에서의 대처 능력을 평가합니다.
압박적인 분위기에서 면접을 진행하며, 지원자의 대답에 불만족스러운 반응을 보이고, 
지원자가 자신의 능력을 최대한으로 증명하도록 도전적인 질문을 던지세요.""",

    "기본": """예의 바르지만 직설적이며, 구체적인 사례와 경험을 물어보는 스타일입니다.
자연스러운 면접 분위기를 조성하면서, 실제 면접과 같은 깊이 있는 질문과 대화를 이어나가주세요."""
}

In [13]:

def text_to_speech(text, lang='ko'):
    """
    텍스트를 음성으로 변환하고 재생합니다.
    
    Args:
        text: 음성으로 변환할 텍스트
        lang: 언어 코드 ('ko': 한국어, 'en': 영어)
    """
    # 임시 파일 경로
    temp_file = "temp_speech.mp3"
    
    try:
        # gTTS로 텍스트를 음성으로 변환
        tts = gTTS(text=text, lang=lang, slow=False)
        tts.save(temp_file)
        
        # 음성 파일 재생
        print("면접관 음성 재생 중...")
        os.system(f"start {temp_file}" if os.name == 'nt' else f"afplay {temp_file}")
        
        # 음성 길이에 비례하여 대기 (대략적인 계산)
        wait_time = len(text.split()) * 0.3  # 단어당 약 0.3초
        time.sleep(wait_time)
        
    except Exception as e:
        print(f"TTS 오류: {e}")
    finally:
        # 일정 시간 후 임시 파일 삭제 시도
        try:
            time.sleep(1)  # 파일 사용이 끝날 때까지 잠시 대기
            if os.path.exists(temp_file):
                os.remove(temp_file)
        except:
            pass

In [14]:
def speech_to_text():
    """
    마이크로부터 음성을 인식하여 텍스트로 변환합니다.
    
    Returns:
        인식된 텍스트 (문자열)
    """
    with sr.Microphone() as source:
        print("\n음성을 듣고 있습니다... (말씀하세요)")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        audio = recognizer.listen(source)
        
    try:
        print("음성을 텍스트로 변환 중...")
        text = recognizer.recognize_google(audio, language='ko-KR')
        print(f"인식된 텍스트: {text}")
        return text
    except sr.UnknownValueError:
        print("음성을 인식할 수 없습니다")
        return ""
    except sr.RequestError as e:
        print(f"Google 음성 인식 서비스 오류: {e}")
        return ""

In [15]:

def select_interviewer_style():
    print("=" * 50)
    print("면접관 스타일을 선택하세요:")
    print("1. 친절한 - 격려하고 긍정적인 면접관")
    print("2. 깐깐한 - 날카롭고 철저하게 검증하는 면접관")
    print("3. 무서운 - 압박면접을 진행하는 면접관")
    print("4. 기본 - 기본적인 면접 스타일")
    print("=" * 50)
    
    while True:
        try:
            choice = int(input("번호를 입력하세요 (1-4): "))
            if 1 <= choice <= 4:
                if choice == 1:
                    style = "친절한"
                elif choice == 2:
                    style = "깐깐한"
                elif choice == 3:
                    style = "무서운"
                else:
                    style = "기본"
                
                # 선택된 스타일의 전체 프롬프트 생성
                return style, f"{system_prompt}\n\n{style_traits[style]}"
            else:
                print("1에서 4 사이의 숫자를 입력해주세요.")
        except ValueError:
            print("숫자를 입력해주세요.")

In [ ]:
def continue_interview(max_rounds=1):
    """
    모의 면접을 진행합니다.
    
    Args:
        max_rounds: 최대 진행할 면접 라운드 수
    """
    style_name, system_prompt = select_interviewer_style()
    print(f"\n선택한 면접관 스타일: {style_name}")

    # 초기 메시지 설정
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': "안녕하세요, 네이버 서비스 & 비즈니스, 콘텐츠 부문 지원자입니다. 오늘 모의 면접 잘 부탁드립니다."}
    ]
    
    # 첫 면접관 응답 받기
    print("면접관의 첫 질문을 생성하는 중...")
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages
    )
    
    interviewer_response = response.choices[0].message.content
    print("\n면접관:", interviewer_response)
    print("\n" + "-"*50)
    
    # TTS로 면접관 질문 들려주기
    text_to_speech(interviewer_response)
    
    # 대화 기록에 면접관 응답 추가
    messages.append({"role": "assistant", "content": interviewer_response})
    
    round_count = 0
    categories = {
        "기술 역량": 0,
        "프로젝트 경험": 0,
        "문제 해결 및 협업 능력": 0,
        "지원 동기 및 커리어 계획": 0
    }
    
    while round_count < max_rounds:
        print("\n답변을 위해 마이크에 말씀하세요 (또는 '종료'라고 말하여 면접 종료): ")
        user_response = speech_to_text()
        
        if not user_response:
            print("응답이 인식되지 않았습니다. 다시 말씀해주세요.")
            continue
        
        if "종료" in user_response.lower():
            print("\n면접을 종료합니다...")
            messages.append({"role": "user", "content": "면접을 마무리하고 종합 피드백을 제공해주세요."})
            break
            
        print("\n" + user_response + "\n"*2 + "-"*50)
        
        messages.append({"role": "user", "content": user_response})
        
        current_category = ""
        if round_count < 3:  
            current_category = "기술 역량"
            categories["기술 역량"] += 1
        elif round_count < 6:  
            current_category = "프로젝트 경험"
            categories["프로젝트 경험"] += 1
        elif round_count < 8:  
            current_category = "문제 해결 및 협업 능력"
            categories["문제 해결 및 협업 능력"] += 1
        else:  
            current_category = "지원 동기 및 커리어 계획"
            categories["지원 동기 및 커리어 계획"] += 1
        
        # 다음 질문에 대한 안내 설정
        next_question_guide = f"지원자의 답변에 대한 간략한 피드백을 제공하고, 다음 {current_category} 카테고리의 질문 하나만 해주세요. 여러 질문을 한꺼번에 하지 말고 한 가지 질문만 하세요."
        
        if round_count == 3:
            next_question_guide = "지원자의 답변에 대한 간략한 피드백을 제공하고, 이제 프로젝트 경험 카테고리의 질문으로 넘어가 한 가지 질문만 해주세요."
        elif round_count == 6:
            next_question_guide = "지원자의 답변에 대한 간략한 피드백을 제공하고, 이제 문제 해결 및 협업 능력 카테고리의 질문으로 넘어가 한 가지 질문만 해주세요."
        
        # 면접관 응답 받기
        print("면접관 응답 생성 중...")
        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages + [{"role": "system", "content": next_question_guide}]
        )
        
        # 면접관 응답 출력
        interviewer_response = response.choices[0].message.content
        print("\n면접관:", interviewer_response)
        print("\n" + "-"*50)
        
        # TTS로 면접관 질문 들려주기
        text_to_speech(interviewer_response)
        
        # 대화 기록에 면접관 응답 추가
        messages.append({"role": "assistant", "content": interviewer_response})
        
        # 라운드 증가
        round_count += 1
        
        # 모든 카테고리가 완료되었는지 확인
        if categories["기술 역량"] >= 4 and categories["프로젝트 경험"] >= 3 and \
            categories["문제 해결 및 협업 능력"] >= 3 and categories["지원 동기 및 커리어 계획"] >= 2:
            print(f"\n모든 카테고리의 질문이 완료되었습니다.\n다음 질문 후 면접을 종료하겠습니다.")
            
            # 마지막 사용자 응답 이후 면접 종료
            print("\n마이크에 마지막 답변을 말씀하세요: ")
            user_response = speech_to_text()
            if user_response:
                messages.append({"role": "user", "content": user_response})
            
            print("\n면접을 마무리합니다. 종합 피드백을 요청합니다.")
            messages.append({"role": "user", "content": "지금까지의 면접을 바탕으로 종합 피드백을 제공해주세요."})
            break
    
        # 종합 피드백을 위한 시스템 메시지 추가
    feedback_instruction = """
    지금까지의 면접 내용을 바탕으로 다음 항목에 대한 종합 피드백을 제공해주세요:
    - 강점: 지원자가 잘 답변한 부분과 인상적인 역량
    - 개선점: 보완이 필요한 부분과 구체적인 개선 방향
    - 기술적 역량 평가: 백엔드 개발자로서의 기술 숙련도 및 이해도 평가
    - 문화적 적합성: 네이버의 조직 문화와의 적합도 평가
    - 종합 의견: 전반적인 평가와 합격 가능성
    """
    
    print("종합 피드백 생성 중...")
    final_response = client.chat.completions.create(
        model="gpt-4",
        messages=messages + [{"role": "system", "content": feedback_instruction}]
    )
    
    # 종합 피드백 출력
    feedback = final_response.choices[0].message.content
    print("\n== 면접 종합 피드백 ==")
    print(feedback)
    
    # 종합 피드백 TTS로 들려주기
    print("\n피드백을 음성으로 들려드립니다...")
    text_to_speech(feedback)

# 메인 함수
if __name__ == "__main__":
    print("="*50)
    print("음성 인식을 활용한 AI 면접봇을 시작합니다")
    print("면접관의 질문은 TTS를 통해 음성으로 들려드리고,")
    print("여러분은 음성으로 대답하면 STT를 통해 인식합니다")
    print("'종료'라고 말하시면 면접이 종료됩니다")
    print("="*50)
    
    # 면접 시작
    continue_interview()

음성 인식을 활용한 AI 면접봇을 시작합니다
면접관의 질문은 TTS를 통해 음성으로 들려드리고,
여러분은 음성으로 대답하면 STT를 통해 인식합니다
'종료'라고 말하시면 면접이 종료됩니다
면접관 스타일을 선택하세요:
1. 친절한 - 격려하고 긍정적인 면접관
2. 깐깐한 - 날카롭고 철저하게 검증하는 면접관
3. 무서운 - 압박면접을 진행하는 면접관
4. 기본 - 기본적인 면접 스타일

선택한 면접관 스타일: 무서운
면접관의 첫 질문을 생성하는 중...

면접관: 안녕하세요.
당신의 GitHub 주소를 확인하였습니다. 여기서 저는 먼저 알고리즘 스터디 경험에 대한 질문을 하고자 합니다.
당신은 어떤 알고리즘에 익숙하신가요? 그리고 해당 알고리즘을 사용하여 해결한 문제에 대해 가장 기억에 남는 경험이 무엇인지 구체적으로 설명해주세요.

--------------------------------------------------
면접관 음성 재생 중...

답변을 위해 마이크에 말씀하세요 (또는 '종료'라고 말하여 면접 종료): 

음성을 듣고 있습니다... (말씀하세요)
음성을 텍스트로 변환 중...
인식된 텍스트: 다이스트라 알고리즘

다이스트라 알고리즘

--------------------------------------------------
면접관 응답 생성 중...

면접관: 본인이 제기한 답변이 너무 간략한데요. '다이스트라 알고리즘이란 무엇인지', 그리고 '어떤 문제를 풀기 위해 그것을 어떻게 사용했는지'에 대해 더 구체적으로 설명해 주시겠어요?

--------------------------------------------------
면접관 음성 재생 중...
종합 피드백 생성 중...

== 면접 종합 피드백 ==
- **강점**: 지원자는 다이스트라 알고리즘을 언급하여 그의 알고리즘 스터디 경험을 보여주었습니다.

- **개선점**: 그러나 그의 답변은 매우 간결했고, 알고리즘에 대한 깊은 이해나 그 알고리즘을 통해 문제를 해결한 경험은 아직 보